In [90]:
# imports
import os
import io
import tokenize
import numpy as np
import pandas as pd

In [3]:
# paths
zeppelin_folder = '/home/ourownstory/Documents/SOL/zeppelin-solidity-master/contracts/token/'
# os.listdir(zeppelin_folder)
# BasicToken.sol     CappedToken.sol    ERC20Basic.sol  MintableToken.sol  SafeERC20.sol      TokenTimelock.sol
# BurnableToken.sol  DetailedERC20.sol  ERC20.sol       PausableToken.sol  StandardToken.sol  TokenVesting.sol

In [19]:
# read in as txt
raw_lines = []
with open(zeppelin_folder + 'BasicToken.sol', 'r') as file:
    for line in file:
        raw_lines.append(line)

In [20]:
# raw_lines

In [21]:
for raw_line in raw_lines:
    print(raw_line.split())

['pragma', 'solidity', '^0.4.18;']
[]
[]
['import', "'./ERC20Basic.sol';"]
['import', "'../math/SafeMath.sol';"]
[]
[]
['/**']
['*', '@title', 'Basic', 'token']
['*', '@dev', 'Basic', 'version', 'of', 'StandardToken,', 'with', 'no', 'allowances.']
['*/']
['contract', 'BasicToken', 'is', 'ERC20Basic', '{']
['using', 'SafeMath', 'for', 'uint256;']
[]
['mapping(address', '=>', 'uint256)', 'balances;']
[]
['/**']
['*', '@dev', 'transfer', 'token', 'for', 'a', 'specified', 'address']
['*', '@param', '_to', 'The', 'address', 'to', 'transfer', 'to.']
['*', '@param', '_value', 'The', 'amount', 'to', 'be', 'transferred.']
['*/']
['function', 'transfer(address', '_to,', 'uint256', '_value)', 'public', 'returns', '(bool)', '{']
['require(_to', '!=', 'address(0));']
['require(_value', '<=', 'balances[msg.sender]);']
[]
['//', 'SafeMath.sub', 'will', 'throw', 'if', 'there', 'is', 'not', 'enough', 'balance.']
['balances[msg.sender]', '=', 'balances[msg.sender].sub(_value);']
['balances[_to]', '=', '

In [22]:
# would need to use hand-crafted regex to split as wanted.
# TODO: implement if tokenize not desirable.

In [24]:
# approach using tokenize


In [106]:
tokens_dict = {
    'line': [],
    'type': [],
    'value': [],
}
tokens_list = []
with open(zeppelin_folder + 'BasicToken.sol', 'r') as file:
    token_generator = tokenize.generate_tokens(file.readline)
    for type, token, (srow, scol), (erow, ecol), line in token_generator:
        tokens_dict['line'].append(srow)        
        tokens_dict['type'].append(tokenize.tok_name[type])        
        tokens_dict['value'].append(repr(token))
        tokens_list.append([srow, tokenize.tok_name[type], repr(token)])
        
tokens = pd.DataFrame.from_dict(tokens_dict)

In [107]:
tokens

,line,type,value
0,1,NAME,'pragma'
1,1,NAME,'solidity'
2,1,OP,'^'
3,1,NUMBER,'0.4'
4,1,NUMBER,'.18'
5,1,OP,';'
6,1,NEWLINE,'\n'
7,2,NL,'\n'
8,3,NL,'\n'
9,4,NAME,'import'


In [109]:
# token_types, type_counts = np.unique(tokens['type'], return_counts=True)
# print(token_types,type_counts)

token_type_freq = pd.Series.value_counts(tokens.type, sort=True)
print(token_type_freq)

NAME         137
OP           106
NL            34
NEWLINE        8
NUMBER         3
STRING         2
INDENT         1
ENDMARKER      1
DEDENT         1
Name: type, dtype: int64


In [110]:
token_freq = pd.Series.value_counts(tokens.value, sort=True)
print(token_freq[:20])

'\n'          42
'.'           14
';'           12
'*'           11
')'           11
'('           11
'address'      9
'@'            8
'balances'     7
'['            6
'_value'       6
'_to'          6
']'            6
'the'          6
'/'            6
'uint256'      5
'balance'      4
'msg'          4
','            4
'sender'       4
Name: value, dtype: int64


In [114]:
op_freq = pd.Series.value_counts(tokens[tokens.type == 'OP'].value, sort=True)
print(op_freq[:20])

'.'     14
';'     12
'*'     11
'('     11
')'     11
'@'      8
'['      6
']'      6
'/'      6
','      4
'{'      3
'='      3
'**'     3
'}'      3
'<='     1
'//'     1
'>'      1
'^'      1
'!='     1
Name: value, dtype: int64


In [115]:
name_freq = pd.Series.value_counts(tokens[tokens.type == 'NAME'].value, sort=True)
print(name_freq[:20])

'address'     9
'balances'    7
'_value'      6
'the'         6
'_to'         6
'uint256'     5
'sender'      4
'msg'         4
'to'          4
'balance'     4
'dev'         3
'of'          3
'The'         3
'_owner'      3
'param'       3
'transfer'    3
'return'      3
'returns'     2
'import'      2
'function'    2
Name: value, dtype: int64


In [ ]:
## Next steps:

In [ ]:
# 1. write tokenization function that receives a list of paths 
# and does the tokenization and counts for all files in paths.


In [ ]:
# 2. Plot the Data (with matplotlib or port over to R)